In [1]:
import setup

setup.init_django()

In [2]:
from rag import engines, sync

sync.sync_blog_docs()

Syncing 4 docs
Sync done.


In [3]:
query_engine = engines.get_semantic_query_engine()

In [4]:
from rag import embeddings
embedding = embeddings.embed_model
sample_text = "Test text"
embedding_result = embedding.get_text_embedding(sample_text)
print(f"Embedding length: {len(embedding_result)}")

Embedding length: 3072


In [5]:
import psycopg2
from decouple import config

db_url = config("DATABASE_URL_POOL")  # Ensure this matches db.get_database_url()
conn = psycopg2.connect(db_url)
cursor = conn.cursor()
cursor.execute("SELECT embedding FROM public.data_blogpost LIMIT 1")
sample_data = cursor.fetchone()
if sample_data:
    embedding = sample_data[0]
    print(f"Raw embedding type: {type(embedding)}")
    print(f"Stored embedding length: {len(embedding)}")
else:
    print("No data found in table")
cursor.close()
conn.close()

Raw embedding type: <class 'str'>
Stored embedding length: 39410


In [6]:
response = query_engine.query("I am a man")
nodes = response.source_nodes

for node in response.source_nodes:
    print("Score", node.score)
    print("Contents", node.node.get_content())
    print("Metadata", node.node.metadata)

Score 0.6710816825725702
Contents The man is dark and woman is light
Metadata {'pk': 273, 'title': 'Blog Post 4'}
Score 0.6573250677175947
Contents The man is looking at the woman
Metadata {'pk': 270, 'title': 'Blog Post 1'}


In [7]:
query_retriever_engine = engines.get_semantic_query_retriever_engine(top_k=5)

In [8]:
ranked_response = query_retriever_engine.query("The sun is hot")

In [9]:
for node in ranked_response.source_nodes:
    print("Score", node.score)
    print("Contents", node.node.get_content())
    print("Metadata", node.node.metadata)

Score 0.7174792503559445
Contents The day is bright
Metadata {'pk': 272, 'title': 'Blog Post 3'}
Score 0.633011064945993
Contents The man is dark and woman is light
Metadata {'pk': 273, 'title': 'Blog Post 4'}
Score 0.6065898952904399
Contents The man is looking at the woman
Metadata {'pk': 270, 'title': 'Blog Post 1'}
Score 0.6052388784146988
Contents The woman is looking at the man
Metadata {'pk': 271, 'title': 'Blog Post 2'}


In [8]:
# from llama_index.core import Settings
# from rag.engines import get_vector_store

# vector_store = get_vector_store()
# embeddings = [Settings.embed_model.get_text_embedding(text) for text in ["Test text"]]
# for emb in embeddings:
#     print(f"Embedding type before storage: {type(emb)}")
#     print(f"Embedding length before storage: {len(emb)}")  # Should be 3072
# # Simulate insertion (adjust based on your indexing logic)